# Feature Extractor

The feature extractor within person-reid is an API that shows the embeddings produced by the learned model. In doing so, lets us see the features it finds important and can be built off of a prebuilt model. This notebook serves an example of how the feature extractor is used.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/person-reid/deep-person-reid

/content/drive/MyDrive/person-reid/deep-person-reid


In [ ]:
!conda create --name torchreid python=3.7
!conda activate torchreid
!pip install -r requirements.txt
!conda install pytorch torchvision cudatoolkit=10.2 -c pytorch
!python setup.py develop

In [ ]:
from comp_vis_data import CvDataSet

In [ ]:
import torchreid

torchreid.data.register_image_dataset('cvision_dataset', CvDataSet)

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='cvision_dataset',
    height=256,
    width=128
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
self.dataset_dir /content/drive/My Drive/person-reid/deep-person-reid/reid-data/data
=> Loaded CvDataSet
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |     2 |      169 |         7
  query    |   

Build off a pre-built model, in this case we use the duke 256x128 model to make feature predictions

In [ ]:
import os
path = os.getcwd() + '/pre-trained-models/osnet_ibn_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter.pth'
path

'/content/drive/My Drive/person-reid/deep-person-reid/pre-trained-models/osnet_ibn_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter.pth'

In [ ]:
!python tools/visualize_actmap.py \
--root testing_data/ \
-d market1501 \
-m osnet_x1_0 \
--weights path \
--save-dir log/visactmap_osnet_x1_0_market1501xDuke

/usr/local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370128159/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/drive/My Drive/person-reid/deep-person-reid/t

This model uses the osnet model with the pretrained model on market1501 with softmax and label smoothing to query extract features from the list of data we provide from our dataset. (Our data is located within cv_dataset, we use the query function to see what the embeddings it produces are).

In [ ]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path='osnet_ibn_x1_0_market1501_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter.pth',
    device='cuda'
)

image_list = [
    './reid-data/data/cv_dataset/query/0006/0006_c2s1_10.jpg',
]

features = extractor(image_list)
print(features.shape) # output (5, 512)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352


/content/drive/MyDrive/person-reid/deep-person-reid/torchreid/utils/tools.py:42: UserWarning: No file found at "osnet_ibn_x1_0_market1501_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip_jitter.pth"
  warnings.warn('No file found at "{}"'.format(fpath))


torch.Size([1, 512])


This is the list of features that it learns in the embedding layer that uses euclidean distance to find similarity in images

In [ ]:
features

tensor([[8.2457e-01, 2.6057e+00, 0.0000e+00, 0.0000e+00, 6.5838e-01, 5.1139e-01,
         0.0000e+00, 9.7932e-01, 6.2505e-01, 1.2671e-01, 0.0000e+00, 1.8568e+00,
         0.0000e+00, 0.0000e+00, 1.4102e+00, 0.0000e+00, 1.2224e+00, 1.3301e+00,
         1.5735e-01, 0.0000e+00, 2.1626e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 6.8913e-01, 1.3931e+00, 1.7769e+00, 0.0000e+00, 0.0000e+00,
         1.5195e+00, 1.5002e+00, 0.0000e+00, 5.6601e-01, 1.4144e+00, 0.0000e+00,
         0.0000e+00, 1.6008e+00, 1.8294e+00, 3.6606e+00, 1.2931e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6254e-01, 1.1560e+00, 2.6188e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 8.4930e-01, 1.8764e-01, 7.2190e-01,
         3.3500e+00, 2.3105e+00, 1.1536e+00, 2.1683e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 9.0640e-01, 0.0000e+00, 2.8864e-01, 1.6341e+00, 0.0000e+00,
         3.8130e+00, 7.1817e-01, 0.0000e+00, 0.0000e+00, 2.3098e-03, 3.9842e-02,
         0.0000e+00, 0.0000e